# Load CATMAID Gap Junctions

#### Initialize CATMAID Helper Class

In [6]:
import sys
import pandas as pd
sys.path.append("../catmaid")
from catmaidhelper import CatmaidHelper

pid = 290 #SEM Adult, 301 for dauer1 290 for dauer2, 281 for adult

with open("/home/tommytang111/gap-junction-segmentation/secrets.txt") as f:
    api_token = f.readlines()[1].strip()
url = 'https://zhencatmaid.com/'

catmaid = CatmaidHelper(url, api_token)
catmaid.set_project(pid)

# skeletons = catmaid.get_skeletons()
# skeleton_names = catmaid.load_skeleton_names(skeletons)

Gap Junctions are a subset of connector objects in CATMAID. \
Getting a GJ Connector gives you the GJ, and the partners as a skeleton. \
Need to make sure relation_name is "gapjunction_with" for the connector to extract GJs.

In [7]:
#Connector endpoint for fetching all connectors in a project
connectors = catmaid.fetch(
    url = f"/{pid}/connectors/",
    method = "post",
    data = {
        "project_id": pid,
    }).json()['connectors']

cid = connectors[0][0]

In [165]:
#Connector endpoint for fetching a specific connector by its ID
connector_info = catmaid.fetch(
    url = f"/{pid}/connectors/{cid}/",
    method = "get",
    data = {
        "project_id": pid,
        "connector_id": cid
    }).json()

In [170]:
'x' in connector_info.keys()

True

In [ ]:
#Make a dictionary of gap junction connectors and their coordinates
#Coordinates are in nm, not voxels, so will need to be converted later
gap_junction_dict = {}
for connector in tqdm(connectors, total=len(connectors)):
    cid = connector[0]
    #Get connector info for each connector ID
    connector_info = catmaid.fetch(
    url = f"/{pid}/connectors/{cid}/",
    method = "get",
    data = {"project_id": pid, "connector_id": cid}).json()
    if 'detail' not in connector_info.keys():
        if connector_info['partners'][0]['relation_name'] == "gapjunction_with" and connector_info['partners'][1]['relation_name'] == "gapjunction_with":
            gap_junction_dict[cid] = [connector[1:4]]

100%|██████████| 14641/14641 [10:52<00:00, 22.45it/s]


In [174]:
len(gap_junction_dict)

8145

In [175]:
#Save gap junction dict as json
import json 

with open("/home/tommytang111/gap-junction-segmentation/sem_dauer_2_GJs.json", "w") as f:
    json.dump(gap_junction_dict, f)

### Convert JSON to NPY

In [1]:
import sys
import numpy as np
sys.path.insert(0, '/home/tommytang111/gap-junction-segmentation/code/src')
from utils import split_img
import pandas as pd
import numpy as np
import cv2 

#Load points and get dimensions of a section
gj_points = pd.read_json("/home/tommytang111/gap-junction-segmentation/gj_point_annotations/sem_adult_GJs.json", orient='index')
gj_points.rename(columns={0: "points"}, inplace=True)
shape = cv2.imread("/mnt/e/Tommy/SEM_adult/Predictions/2D/SEM_adult_image_export_s250_pred.png", cv2.IMREAD_GRAYSCALE).shape
print(f"Section Shape: {cv2.imread('/mnt/e/Tommy/SEM_adult/Sections/SEM_adult_image_export_s250.png', cv2.IMREAD_GRAYSCALE).shape}")
print(f"Prediction Shape: {shape}")

Section Shape: (7000, 10500)
Prediction Shape: (7000, 10500)


In [2]:
#Get max coordinates in annotations for a particular dataset
max_x, max_y, max_z = 0, 0, 0
for gj in gj_points['points']:
    if gj[0] > max_x:
        max_x = gj[0]
    if gj[1] > max_y:
        max_y = gj[1]
    if gj[2] > max_z:
        max_z = gj[2]

In [3]:
#See if GJ points dataset dimensions match dataset I'm working with
print(max_x, max_y, max_z)  #10500, 7000, 700 something like this shape
print(max_x//4, max_y//4, max_z//30)

72776.2 37034.0 21420.0
18194.0 9258.0 714.0


In [10]:
test_df = gj_points.apply(lambda row: (row['points'][0]//4, row['points'][1]//4, row['points'][2]//30), axis=1)
split_df=pd.DataFrame()
split_df[['x', 'y', 'z']] = pd.DataFrame(test_df.tolist(), index=test_df.index)

In [ ]:
#DON'T ACTUALLY RUN THIS, WILL GET OOM ERROR. Implementation is in transforms/make_point_volume_from_JSON.py
#Create empty_volume
test_vol = np.zeros((972, 8328, 9360), dtype=np.uint8)  # Z, Y, X

#Populate volume with gap junction points
for gj in gj_points['points']:
    test_vol[int(gj[2])//50, int(gj[1])//2, int(gj[0])//2] = 255 
    
#Save volume
np.save("/home/tommytang111/gap-junction-segmentation/gj_point_annotations/sem_adult_GJs.npy", test_vol)

### William

In [ ]:
#William's stuff
import pandas as pd
from tqdm import tqdm
import concurrent.futures as futures

# Load all connector info

col_names = ["connector_id","x","y","z"]
col_ids = [0,1,2,3]

connectors = pd.DataFrame(catmaid.get_connectors()).iloc[:,col_ids]
connectors = connectors.rename(columns=dict(zip(col_ids,col_names)))

def get_link_info(connector_id):
    connector_subtable = []
    connector_info = catmaid.get_connector_info(connector_id)
    pre = ""
    pre_node = 0
    post = []
    post_nodes = []
    for partner in connector_info['partners']:
        if partner['relation_name'] == 'presynaptic_to':
            if partner['confidence'] < 5:
                continue
            pre = partner['skeleton_id']
            pre_node = partner['partner_id']
        if partner['relation_name'] == 'postsynaptic_to':
            if partner['confidence'] < 5:
                continue
            post.append(partner['skeleton_id'])
            post_nodes.append(partner['partner_id'])
    for i in range(len(post)):
        connector_subtable.append({
            "connector_id": connector_info['connector_id'],
            "x": connector_info['x'],
            "y": connector_info['y'],
            "z": connector_info['z'],
            "pre": pre,
            "pre_node": pre_node,
            "post": post[i],
            "post_node": post_nodes[i]})
    return connector_subtable

with futures.ThreadPoolExecutor() as executor:
    results = list(tqdm(executor.map(get_link_info, connectors['connector_id']), total=len(connectors)))

connector_table = [item for sublist in results if sublist for item in sublist]
connector_table = pd.DataFrame(connector_table)

# only include elements within bounded nodes for each particular skeleton
pre_skel_filt = connector_table['pre'].isin(bounded_nodes.keys())
connector_table = connector_table[pre_skel_filt]

all_bounded_nodes = set()
for nodes in bounded_nodes.values():
    all_bounded_nodes.update(nodes)

connector_table_filt = connector_table[connector_table['pre_node'].isin(all_bounded_nodes)]

100%|██████████| 4455/4455 [02:50<00:00, 26.12it/s]
